# Behavioural Analysis

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('allDataCleaned_analysis - shared.csv')
df

,SLNum,SchoolNum,1_Timestamp,2_Your Name,4_Gender,6_School Name,7_Grade,8_Age,9_Teacher,10_One Interesting thing about yourself,...,44_None,21_ReportedProblemTo,TypeOfProduct,ProblemsSolved,ProblemsSolved_Group,Group,City,School Type,Unnamed: 110,Unnamed: 111
0,1,11.0,43290.52383,Anu R,NaN,GHPS Police colony,6,11,Kaivalya,To study,...,NaN,Admin (Govt),NaN,1,One,Govt School,Bangalore,Govt School,NaN,NaN
1,2,11.0,43290.52543,Sabareddy,NaN,GHPS Police colony,6,12,Kaivalya,NaN,...,NaN,NaN,NaN,0,None,Govt School,Bangalore,Govt School,NaN,NaN
2,3,11.0,43290.52815,Sneha,NaN,GHPS Police colony,6,12,Kaivalya,NaN,...,1.0,Admin (School),NaN,1,One,Govt School,Bangalore,Govt School,NaN,NaN
3,4,11.0,43290.53168,Chethana V,NaN,GHPS Police colony,6,10,Kaivalya,I like environment drawing and dance,...,NaN,Admin (School),NaN,2,Two or more,Govt School,Bangalore,Govt School,NaN,NaN
4,5,11.0,43290.53323,Manoj,NaN,GHPS Police colony,6,11,Kaivalya,To practice songs,...,NaN,NaN,NaN,0,None,Govt School,Bangalore,Govt School,NaN,NaN
5,6,11.0,43290.54794,Rathnappa,NaN,GHPS Police colony,6,12,Kaivalya,NaN,...,1.0,Admin (School),NaN,1,One,Govt School,Bangalore,Govt School,NaN,NaN
6,7,11.0,43290.54985,Pallaui G,NaN,GHPS Police colony,6,11,Kaivalya,To study,...,NaN,Admin (School),NaN,1,One,Govt School,Bangalore,Govt School,NaN,NaN
7,8,11.0,43290.55223,Deepak S,NaN,GHPS Police colony,6,11,Kaivalya,I like dramas,...,NaN,NaN,NaN,0,None,Govt School,Bangalore,Govt School,NaN,NaN
8,9,11.0,43290.5545,Gowtham,NaN,GHPS Police colony,6,11,Kaivalya,I like dramas,...,1.0,NaN,NaN,0,None,Govt School,Bangalore,Govt School,NaN,NaN
9,10,11.0,43290.58285,Sripriya S,NaN,GHPS Police colony,6,11,Kaivalya,Group in school,...,NaN,NaN,NaN,0,None,Govt School,Bangalore,Govt School,NaN,NaN


## Cleaning and sanitising the data

In [3]:
# Remove names from school, teacher etc and convert them to categorical variables - 1/2/3/4

for i in df.columns:
    print(i)
    


SLNum
SchoolNum
1_Timestamp
2_Your Name
4_Gender
6_School Name
7_Grade
8_Age
9_Teacher
10_One Interesting thing about yourself
11_Do you know the name of the ward you live in?
12_If yes, please name your ward
13_Do you know the name of your corporator?
14_If yes, please name your corporator 
15_Do you know the name of the constituency you live in? 
16_If yes, please name your constituency
17_Do you know the name of your MLA? 
18_If yes, please name your MLA
19_In the last 6 months, have you reported a problem to an authority
20_How many civic or environmental issues have you tried addressing by reporting an issue
21_If you reported a problem, think of the last problem you reported. Who did you report it to 
22_Please describe the last problem you reported
23_In the last 6 months, have you built or implemented a hands on solution
24_How many civic or environmental issues have you tried addressing by implementing a solution
25_Think of the last environmental or civic solution prototyped 

In [4]:
cols = [ '38_What motivates you to take up the Solve Ninja Program? ', 
'39_What skills are you looking to pick up at the end of this program?',
'40_What are you looking for from Solve Ninja Program?',
'44_What other expectations do you have from this program?' ]

df[cols[3]].dropna()

2                                                    None
5                                                    None
7                                           To solve more
8                                                    None
10                                                   None
11                                                   None
12                                                   None
14                                                   None
15                                                   None
18                                       To bring change 
19                                                   None
20                                        To bring change
21                                       To bring change 
22                                       To bring changes
23                                                   None
24                                                   None
27                                                   None
31          To

## Generating stem words

In [37]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.collocations import *
import nltk.tokenize.punkt
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk.stem.snowball
import string
import nltk
import re

In [38]:
nltk.download('stopwords')
nltk.download('punkt')

# Get default English stopwords and extend otherkey words 
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords.update(string.punctuation)
stemmer = nltk.stem.snowball.SnowballStemmer('english')

#print(stopwords)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aniruddha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aniruddha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
train  = df.dropna(subset=['10_One Interesting thing about yourself'])
train['10_One Interesting thing about yourself']


0                                    To study 
3       I like environment drawing and dance  
4                          To practice songs  
6                                    To study 
7                               I like dramas 
8                               I like dramas 
9                             Group in school 
10                             I like to draw 
12                                     To play
13                              I like dramas 
16                                      Study 
18                                       Dance
20                                       Dance
21                                       Dance
22                                       Dance
39                               Military Army
41                           To read and write
42                             To play kabbadi
43                           To read and write
44                                     To sing
45                  To become a police officer
46           

In [62]:
# Create tokenizer and stemmer
stemmer = nltk.stem.snowball.SnowballStemmer('english')
def tokenize_string(teststr):
    tokens_a = []
    for token in word_tokenize(re.sub('[0-9]+', '', teststr)):
        if token.lower().strip(string.punctuation) not in stopwords:
            tokens_a.append(token.lower().strip(string.punctuation))
        stems_a = [stemmer.stem(token) for token in tokens_a]
    return(stems_a)
        
print(train['10_One Interesting thing about yourself'][3])
print(tokenize_string(train['10_One Interesting thing about yourself'][3]))

I like environment drawing and dance  
['like', 'environ', 'draw', 'danc']


In [63]:
words = []
for row in train['10_One Interesting thing about yourself']:
    words.append(tokenize_string(row))

In [19]:
# Create tokenizer and stemmer
stemmer = nltk.stem.snowball.SnowballStemmer('english')
stems = []
for teststr in df[cols[3]].dropna():
    tokens_a = []
    for token in word_tokenize(re.sub('[0-9]+', '', teststr)):
        if token.lower().strip(string.punctuation) not in stopwords:
            tokens_a.append(token.lower().strip(string.punctuation))

    #print(tokens_a)

    stems_a = [stemmer.stem(token) for token in tokens_a]
    stems+= stems_a


## Word Counts 

In [64]:
words

[['studi'],
 ['like', 'environ', 'draw', 'danc'],
 ['practic', 'song'],
 ['studi'],
 ['like', 'drama'],
 ['like', 'drama'],
 ['group', 'school'],
 ['like', 'draw'],
 ['play'],
 ['like', 'drama'],
 ['studi'],
 ['danc'],
 ['danc'],
 ['danc'],
 ['danc'],
 ['militari', 'armi'],
 ['read', 'write'],
 ['play', 'kabbadi'],
 ['read', 'write'],
 ['sing'],
 ['becom', 'polic', 'offic'],
 ['docter'],
 ['like', 'write', 'stori', 'sing'],
 ['account', 'stude'],
 ['doctor'],
 ['studi'],
 ['cricket'],
 ['studi'],
 ['sing'],
 ['studi'],
 ['build', 'thing'],
 ['studi'],
 ['studi', 'want', 'becom', 'ia', 'offic'],
 ['like', 'read', 'book'],
 ['play', 'sport'],
 ['paint'],
 ['love', 'sing'],
 ['write'],
 ['draw'],
 ['cricket'],
 ['govern', 'doctor'],
 ['teacher'],
 ['danc', '', 'read', '', 'write', 'want', 'becom', 'doctor'],
 ['drama'],
 ['love', 'play', 'cricket'],
 ['listen', 'lectur'],
 ['read', 'write'],
 ['play'],
 ['cricket'],
 ['engin'],
 ['play', 'cricket'],
 ['play', 'cricket'],
 ['drama'],
 ['pl

In [81]:
wordcount = dict()
for wordset in words: 
    for word in wordset:
        if word in wordcount: 
            wordcount[word] += 1
        else:
            wordcount[word] = 1
            
series = pd.Series(wordcount)
series.sort_values(ascending=False)

              279
love          179
like          172
good          129
play          119
footbal        72
help           62
danc           49
read           44
thing          41
friend         40
cricket        38
sport          37
interest       36
make           35
solv           35
draw           31
well           30
peopl          29
problem        29
art            28
sing           26
know           25
other          25
new            25
book           23
want           23
creativ        22
studi          22
lot            22
             ... 
headstrong      1
sam             1
harri           1
sarcasm         1
hands-on        1
human           1
handl           1
scientif        1
hair            1
hack            1
gymnast         1
gym             1
row             1
roman           1
helpful         1
hide            1
hindi           1
hippo           1
right           1
hod             1
hold            1
holder          1
reus            1
hors            1
hot       

## testing and experimenting  section ( ignore)

In [16]:
import string
train['tokenize']=train['10_One Interesting thing about yourself'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
train['tozenize']=train['tokenize'].apply(lambda x: x.lower())
train['tokenz'] = train['tokenize'].apply(lambda x: x.split())
train['tokenz'].head()

/home/aniruddha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/aniruddha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/aniruddha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

0                                    [To, study]
3    [I, like, environment, drawing, and, dance]
4                          [To, practice, songs]
6                                    [To, study]
7                              [I, like, dramas]
Name: tokenz, dtype: object

In [17]:
import itertools

#this will create a list of all words
words=list(itertools.chain.from_iterable(train['tokenz']))

#this will create a list of unique words
unique_words=list(set(words))

print('Number of unique words:',len(set(unique_words)))

Number of unique words: 1418


In [18]:
counts = pd.DataFrame(0,index=np.arange(train.shape[0]), columns=unique_words)


In [20]:
#now counting the number of words in each headline and adding it to our dataframe
for index, row in train.iterrows():
    for token in row['tokenz']:
        try:
            counts.iloc[index][token]+=1
        except:
            continue

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X=vectorizer.fit_transform(list(train['10_One Interesting thing about yourself']))
counts=pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())

In [24]:
count_sum=counts.sum()
counts=counts.drop(count_sum[(count_sum>100) | (count_sum<5)].index,axis=1)

In [65]:
train['10_One Interesting thing about yourself']

0                                    To study 
3       I like environment drawing and dance  
4                          To practice songs  
6                                    To study 
7                               I like dramas 
8                               I like dramas 
9                             Group in school 
10                             I like to draw 
12                                     To play
13                              I like dramas 
16                                      Study 
18                                       Dance
20                                       Dance
21                                       Dance
22                                       Dance
39                               Military Army
41                           To read and write
42                             To play kabbadi
43                           To read and write
44                                     To sing
45                  To become a police officer
46           